In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

## Read CSV into pd.DataFrame

In [3]:
df = pd.read_csv('monthly_data/rail-jul.csv')

## Convert Datetime to a datetime object in a column named datetime

In [4]:
df['datetime'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df.drop(columns = ['timestamp'], inplace=True)

## Reorder DF so rows apear in chronological order

In [5]:
df.sort_values('datetime', inplace=True)

## Reindex while dropping the old 

In [6]:
df.reset_index(drop=True, inplace=True)

In [7]:
df.head()

,bearing,blockID,delay,direction,garage,lastLocID,lastStopSeq,messageCode,nextLocID,nextStopSeq,routeNumber,signMessageLong,tripID,type,vehicle_id,vehicle_location_longitude,vehicle_location_latitude,datetime
0,20,9066,-87,0,RUBY,13123,3,972,7777,4,190,MAX Yellow Line to Expo Center,7429389,rail,225,-122.679549,45.517414,2017-07-01 00:00:00
1,20,9066,-87,0,RUBY,13123,3,972,7777,4,190,MAX Yellow Line to Expo Center,7429389,rail,315,-122.679549,45.517414,2017-07-01 00:00:00
2,319,9070,50,0,RUBY,13720,1,970,13721,2,290,MAX Orange Line to City Center/Expo Center,7431883,rail,227,-122.639114,45.436413,2017-07-01 00:00:01
3,350,9083,0,1,RUBY,13132,24,978,13132,1,200,MAX Green Line to City Center/PSU,7431229,rail,416,-122.567831,45.435793,2017-07-01 00:00:01
4,0,9082,8,1,RUBY,13139,8,978,8370,9,200,MAX Green Line to City Center/PSU,7431227,rail,403,-122.564359,45.515267,2017-07-01 00:00:01


## df[0:1] are copies with different vehicle_id's

### Save the shape of the df before removing duplicate values

In [8]:
original =  df.shape[0]

## Identify duplicates

In [9]:
df[['tripID', 'datetime', 'vehicle_id']].groupby(['tripID', 'datetime']).nunique()[df[['tripID', 'datetime', 'vehicle_id']].groupby(['tripID', 'datetime']).nunique()['vehicle_id']>1].head()

tripID  datetime  vehicle_id
tripID  datetime                                         
7427095 2017-07-01 00:00:15       1         1           2
        2017-07-01 00:02:25       1         1           2
        2017-07-01 00:04:53       1         1           2
        2017-07-01 00:07:00       1         1           2
7427096 2017-07-01 00:00:08       1         1           2

## Before the duplicates can be droped the df must be 'subordered' by vehicle_id so the same vehicle is always dropped from the duplicates

In [10]:
df.sort_values(by=['datetime', 'vehicle_id'], inplace=True)

In [11]:
df[['tripID', 'datetime', 'vehicle_id', 'lastLocID']].loc[df['tripID'].isin([7427095])]

,tripID,datetime,vehicle_id,lastLocID
75,7427095,2017-07-01 00:00:15,230,8347
76,7427095,2017-07-01 00:00:15,327,8347
186,7427095,2017-07-01 00:02:25,230,10572
187,7427095,2017-07-01 00:02:25,327,10572
284,7427095,2017-07-01 00:04:53,230,10574
285,7427095,2017-07-01 00:04:53,327,10574
372,7427095,2017-07-01 00:07:00,230,10576
371,7427095,2017-07-01 00:07:00,327,10576


## Drop the duplicated trains to a new df called ndf

In [12]:
ndf = df.drop_duplicates(subset=['tripID', 'datetime', 'lastLocID']).copy()

In [13]:
ndf[['tripID', 'datetime', 'vehicle_id', 'lastLocID']].loc[df['tripID'].isin([7427095])]

,tripID,datetime,vehicle_id,lastLocID
75,7427095,2017-07-01 00:00:15,230,8347
186,7427095,2017-07-01 00:02:25,230,10572
284,7427095,2017-07-01 00:04:53,230,10574
372,7427095,2017-07-01 00:07:00,230,10576


## Success only vehicle_id 230 survives

## How many were dropped?

In [16]:
print(original - ndf.shape[0])
print('{0:.0%}'.format((original - ndf.shape[0])/original))

432105
47%


## Are there any more duplicates?

In [17]:
ndf[['tripID', 'vehicle_id']].groupby(['tripID']).nunique()[ndf[['tripID', 'vehicle_id']].groupby(['tripID']).nunique()['vehicle_id']>1].head()

,tripID,vehicle_id
tripID,,
7427104,1,2
7427105,1,2
7427115,1,2
7427120,1,2
7427132,1,2


In [19]:
ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].loc[ndf['tripID'].isin([7427104])].groupby(['lastLocID']).count()

,tripID,vehicle_id,datetime
lastLocID,,,
8333,1,1,1
8334,1,1,1
8335,1,1,1
8336,1,1,1
8337,1,1,1
8338,1,1,1
8339,1,1,1
8340,1,1,1
8341,1,1,1


In [22]:
ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].loc[ndf['tripID'].isin([7427104])].groupby(['lastLocID']).count()[ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].loc[ndf['tripID'].isin([7427104])].groupby(['lastLocID']).count()['vehicle_id']>1]



,tripID,vehicle_id,datetime
lastLocID,,,
9758,2,2,2


In [23]:
ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].loc[(ndf['tripID'].isin([7427104])) & (ndf['lastLocID'].isin([9758]))]

,tripID,vehicle_id,datetime,lastLocID
4449,7427104,327,2017-07-01 01:57:37,9758
4450,7427104,230,2017-07-01 01:57:38,9758


## It apears that when the train arrived in the station the time recorded for some reason was +1 sec

## First add a column 'day that will be used as an extra column to sort by

#### \*Add day to both dataframes so that the origial df can be grouped by day

#### \*This can also be done with out add the 'day' column by searching with ndf.datetime.dt{}.format(day, month, year, minute, hour, second, etc)

In [24]:
df['day'] = df.datetime.apply(lambda x: x.day)
ndf['day'] = ndf.datetime.apply(lambda x: x.day)

In [27]:
ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].groupby([ndf.tripID, ndf.datetime.dt.minute, ndf.datetime.dt.day, ndf.lastLocID]).nunique()[ndf[['tripID', 'vehicle_id', 'datetime', 'lastLocID']].groupby([ndf.tripID, ndf.datetime.dt.minute, ndf.datetime.dt.day, ndf.lastLocID]).nunique()['vehicle_id']>2]

,,,,tripID,vehicle_id,datetime,lastLocID
tripID,datetime,datetime,lastLocID,,,,
7511551,47,9,7763,1,3,3,1
7512345,21,10,13722,1,3,3,1


In [32]:
ndf.loc[ndf['tripID'].isin([7511551])].loc[ndf['day'].isin([9])][['lastLocID', 'vehicle_id', 'datetime']]

,lastLocID,vehicle_id,datetime
289335,13140,224,2017-07-09 20:23:50
289563,10293,224,2017-07-09 20:34:02
289621,7774,224,2017-07-09 20:36:52
289688,13123,224,2017-07-09 20:39:21
289727,7777,116,2017-07-09 20:41:11
289780,7787,224,2017-07-09 20:43:45
289791,7787,116,2017-07-09 20:44:36
289796,9299,116,2017-07-09 20:45:00
289840,7763,224,2017-07-09 20:47:13
289855,7763,116,2017-07-09 20:47:47


In [33]:
df.loc[df['tripID'].isin([7511551])].loc[df['day'].isin([9])][['lastLocID', 'vehicle_id', 'datetime']]

,lastLocID,vehicle_id,datetime
289335,13140,224,2017-07-09 20:23:50
289563,10293,224,2017-07-09 20:34:02
289621,7774,224,2017-07-09 20:36:52
289688,13123,224,2017-07-09 20:39:21
289727,7777,116,2017-07-09 20:41:11
289726,7777,224,2017-07-09 20:41:11
289780,7787,224,2017-07-09 20:43:45
289791,7787,116,2017-07-09 20:44:36
289796,9299,116,2017-07-09 20:45:00
289840,7763,224,2017-07-09 20:47:13
